![image](images/um_logo.png)

# Computación II


# ***Sockets III***

## Modelo cliente servidor
El modelo cliente-servidor es una arquitectura de diseño utilizada en sistemas de computación y redes para organizar y gestionar la comunicación entre diferentes componentes. En este modelo, los roles principales son desempeñados por dos tipos de entidades: el cliente y el servidor.

- **Cliente**: El cliente es la entidad que solicita algún tipo de servicio o recurso al servidor. Puede ser un programa, una aplicación o incluso un dispositivo que busca obtener información, realizar operaciones o acceder a recursos proporcionados por el servidor. Los clientes envían solicitudes al servidor y esperan respuestas.

- **Servidor**: El servidor es la entidad que proporciona servicios, recursos o información a los clientes. Puede ser un programa, un sistema o un conjunto de hardware que está diseñado para atender las solicitudes de los clientes. El servidor escucha las solicitudes entrantes, procesa la información y envía las respuestas correspondientes.

El modelo cliente-servidor se basa en la idea de división de tareas y responsabilidades. Los clientes solicitan servicios o recursos específicos, mientras que los servidores proveen esos servicios o recursos de manera eficiente y confiable. Esta arquitectura permite una mayor escalabilidad y distribución de tareas, ya que varios clientes pueden acceder a los servicios de un servidor centralizado.

Además del modelo cliente-servidor tradicional, existen varias variantes y enfoques, como el modelo peer-to-peer (P2P), donde las entidades pueden actuar como clientes y servidores al mismo tiempo, compartiendo recursos entre sí sin una jerarquía fija.

El modelo cliente-servidor es ampliamente utilizado en sistemas informáticos y aplicaciones en línea, como navegadores web (cliente) que acceden a sitios web (servidor), aplicaciones de correo electrónico (cliente) que se conectan a servidores de correo, y muchas otras aplicaciones en las que la comunicación y la distribución de recursos son esenciales.

## Programando un servidor

In [ ]:
#!/usr/bin/python3
import socket, os, sys

# create a socket object
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 

# get local machine name
#host = socket.gethostname()
host = ""
#port = int(sys.argv[1])
port = 50001

# bind to the port
serversocket.bind((host, port))

# queue up to 5 requests
serversocket.listen(5)

while True:
    # establish a connection
    clientsocket, addr = serversocket.accept()

    print("Conexión desde %s" % str(addr))

    msg = 'Gracias por conectar'+ "\r\n"
    clientsocket.send(msg.encode('ascii'))
    child_pid = os.fork()
    if not child_pid:
        while True:
            msg = clientsocket.recv(1024)
            print("Recibido: %s" % msg.decode())
            msg = "Ok"+" \r\n"
            clientsocket.send(msg.encode("ascii"))
            
        #clientsocket.close()

¿Qué recursos está usando el servidor de arriba?
El servidor está utilizando, básicamente, sockets y procesos.
Debemos ser cuidadosos a la hora de programar servidores para que los recursos puedan ser liberados correctamente, una vez que se dejaron de utilizar.
Por lo tanto es importante cerrar los sockets. 
También es importante administrar correctamente la finalización de los procesos. Si no lo hago corro riesgo de que se acumulen procesos zombies que puedes, finalmente, sacar de servicio a todo el servidor.
Para hacer esto y, al mismo tiempo, no perder la concurrencia con el uso de wait(), es necesario ignorar la señal SIGCHDL

In [ ]:
import signal

signal.signal(signal.SIGCHLD, signal.SIG_IGN)

https://docs.python.org/3/library/signal.html#signal.SIG_IGN

A continuación se muestra el código de un servidor donde se puede ver de forma más manifiesta el uso de fork.

In [ ]:
#!/usr/bin/python3
import socket, sys, time, os

# create a socket object
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
serversocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
"""
    socket.AF_INET -> sockets tcp/ip
    socket.AF_UNIX -> sockets Unix (archivos en disco, similar a FIFO/named pipes)

    socket.SOCK_STREAM -> socket tcp, orientado a la conexion (flujo de datos)
    socket.SOCK_DGRAM -> socket udp, datagrama de usuario (no orientado a la conexion)
"""

# get local machine name
host = socket.gethostname()                           
#host = ""
#port = int(sys.argv[1])
port = 50001

# bind to the port
serversocket.bind((host, port))                                  

# queue up to 5 requests
serversocket.listen(5)

while True:
    # establish a connection
    print("Esperando conexiones remotas (accept)")
    clientsocket,addr = serversocket.accept()      

    print("Conexión desde %s" % str(addr))
    
    ret = os.fork()
    if not ret: # hijo
        msg = 'Hola Mundo'+ "\r\n"
        #clientsocket.send(msg.encode('ascii'))
        print("Esperando un tiempito...")
        time.sleep(20)
        print("Enviando mensaje...")
        clientsocket.send(msg.encode('utf-8'))
        print("Cerrando conexion...")
        clientsocket.close()
        exit()

Podemos probar el servidor usando telnet y podemos ver las conexiones usando netstat o ss con -npt

## Programando un cliente

In [ ]:
#!/usr/bin/python3

import socket
import sys

# create a socket object
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 

# get local machine name
#host = socket.gethostname()                           
host = sys.argv[1]

port = int(sys.argv[2])

print("Haciendo el connect")
# connection to hostname on the port.
s.connect((host, port))   
print("Handshake realizado con exito!")

# Receive no more than 1024 bytes
print("Esperando datos desde el server")
msg = s.recv(1024)                                     
#print (msg.decode('ascii'))
print (msg.decode('utf-8'))
s.close()
print("Cerrando conexion")

### Analicemos el código
Si intentamos utilizar el cliente que presentamos arriba con alguno de los servidores, el servidor lanza el siguiente error.

¿Qué está ocurriendo?
Arreglemos el código

## Ejemplo de servidor utilizando multiprocessing

In [ ]:
#!/usr/bin/python3
import socket, os, multiprocessing, sys

def mp_server(c):
    print("Launching proc...")
    sock,addr = c
    while True:
        msg = sock.recv(1024)
        print("Recibido: '%s' de %s" % (msg.decode(), addr))
        msg = "Ok"+" \r\n"
        sock.send(msg.encode("ascii"))
        #clientsocket.close()



# create a socket object
# crea un objeto tipo socket para usarlo en la comunicación
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
serversocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# get local machine name
#host = socket.gethostname()
host = ""
port = int(sys.argv[1])

# bind to the port
serversocket.bind((host, port))

# queue up to 5 requests
serversocket.listen(5)

while True:
    # establish a connection
    cliente = serversocket.accept()

    clientsocket, addr = cliente

    print("Got a connection from %s" % str(addr))

    msg = 'Thank you for connecting'+ "\r\n"
    clientsocket.send(msg.encode('ascii'))
    child = multiprocessing.Process(target=mp_server, args=(cliente,))
    child.start()


## Ejemplo de servidor utilizando threading

In [ ]:
#!/usr/bin/python3
import socket, os, threading

def th_server(sock):
    print("Launching thread...")
    while True:
        msg = sock.recv(1024)
        print("Recibido: %s" % msg.decode())
        msg = "Ok"+" \r\n"
        sock.send(msg.encode("ascii"))
        #clientsocket.close()



# create a socket object
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
serversocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# get local machine name
#host = socket.gethostname()
host = ""
port = 1234

# bind to the port
serversocket.bind((host, port))

# queue up to 5 requests
serversocket.listen(5)

while True:
    # establish a connection
    clientsocket,addr = serversocket.accept()

    print("Got a connection from %s" % str(addr))

    msg = 'Thank you for connecting'+ "\r\n"
    clientsocket.send(msg.encode('ascii'))
    th = threading.Thread(target=th_server, args=(clientsocket,))
    th.start()